## API Development For The Fine-Tuned Model

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.0 MB/s eta 0:00:00
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/codeme', methods=['POST'])
def codeme():
    try:
        print("Received POST request to /codeme")

        # Ensure the request has JSON data
        if not request.is_json:
            return {'error': 'Request must contain JSON data'}, 400

        data = request.get_json()

        # Ensure the 'prompt' key is present in the JSON data
        if 'prompt' not in data:
            return {'error': 'Missing "prompt" key in JSON data'}, 400

        prompt = data['prompt']
        print(f"Received prompt: {prompt}")

        # Tokenize input prompt
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        # Generate HTML code
        with torch.no_grad():
            output = model.generate(input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

        # Decode the generated HTML code
        generated_html = tokenizer.decode(output[0], skip_special_tokens=True)

        # Create an HTML page to display the generated HTML code
        html_page = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta http-equiv="X-UA-Compatible" content="IE=edge">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Generated HTML Code</title>
        </head>
        <body>
            <pre>{generated_html}</pre>
        </body>
        </html>
        """

        return html_page, 200, {'Content-Type': 'text/html'}

    except Exception as e:
        print(f"Error: {str(e)}")
        return {'error': str(e)}, 500

if __name__ == '__main__':
  # Add the following import statements for tokenization and model loading
  import torch
  from transformers import AutoTokenizer, AutoModelForCausalLM
  from flask import Flask, request, jsonify
  import torch
  from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

  app = Flask(__name__)


      # from peft import PeftModel
  base_model_id = "HuggingAlgorithms/figr-mistral7b-html"
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  tokenizer = AutoTokenizer.from_pretrained(
      base_model_id,
      add_bos_token=True,
      trust_remote_code=True,
  )# finetuned_model_2 = PeftModel.from_pretrained(model_2, "HuggingAlgorithms/figr-mistral7b-html")

  model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  app.run(host='0.0.0.0', port=8000, debug=True)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
